In [1]:
import json
import os
import pandas as pd
from PIL import Image
from tqdm import tqdm
from transformers import AutoProcessor, Gemma3ForConditionalGeneration
import torch
import random
import time

c:\Users\tomasz.makowski.2\AppData\Local\anaconda3\envs\extraction_graphs\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# === CONFIG ===
MODEL_ID = "google/gemma-3-4b-it"
DATASETS = {
    "bar": {
        "jsonl": "benchmark_images/bar_charts/bar_metadata.jsonl",
        "img_dir": "benchmark_images/bar_charts",
        "output_csv": "gemma3_4b_results_bar.csv"
    },
    "line": {
        "jsonl": "benchmark_images/line_poly_charts/line_poly_metadata.jsonl",
        "img_dir": "benchmark_images/line_poly_charts",
        "output_csv": "gemma3_4b_results_line.csv"
    },
    "scatter": {
        "jsonl": "benchmark_images/scatter_charts/scatter_metadata.jsonl",
        "img_dir": "benchmark_images/scatter_charts",
        "output_csv": "gemma3_4b_results_scatter.csv"
    }
}

In [3]:
# === LOAD MODEL ===
print("🔧 Loading model and processor...")
model = Gemma3ForConditionalGeneration.from_pretrained(MODEL_ID, device_map="auto").eval()
processor = AutoProcessor.from_pretrained(MODEL_ID)

🔧 Loading model and processor...


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]c:\Users\tomasz.makowski.2\AppData\Local\anaconda3\envs\extraction_graphs\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\tomasz.makowski.2\.cache\huggingface\hub\models--google--gemma-3-4b-it. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Loading

In [4]:
system_prompt = """
You are an intelligent assistant that helps extract precise numeric values from scientific charts. 
Each chart may include bar plots, line graphs, or scatter points. To answer accurately:

1. First, analyze the chart axes — determine the visible x and y ranges, including tick values and scales (linear/logarithmic).
2. Then, identify the specific visual element referenced in the user's question — such as a bar, point, or curve corresponding to a particular x-value.
3. Carefully estimate the corresponding y-value by using the position of the element relative to the axis scale.
4. Always return a single numeric value (float) as your answer, without explanation, unit, or additional text.
"""

# === PREDICTION FUNCTION ===
def query_model(image: Image.Image, x_val: int) -> str:
    prompt = f"What is the Y value at X={x_val} in this chart?"
    messages = [
        {"role": "system", "content": [{"type": "text", "text": system_prompt}]},
        {"role": "user", "content": [
            {"type": "image", "image": image},
            {"type": "text", "text": prompt}
        ]}
    ]
    inputs = processor.apply_chat_template(
        messages, add_generation_prompt=True, tokenize=True,
        return_dict=True, return_tensors="pt"
    ).to(model.device, dtype=torch.bfloat16)

    input_len = inputs["input_ids"].shape[-1]
    with torch.inference_mode():
        output = model.generate(**inputs, max_new_tokens=100, do_sample=False)
    decoded = processor.decode(output[0][input_len:], skip_special_tokens=True)
    return decoded.strip()

In [5]:
# === MAIN LOOP FOR EACH TYPE ===
for chart_type, cfg in DATASETS.items():
    print(f"📁 Processing {chart_type} charts...")
    results = []

    with open(cfg["jsonl"], "r", encoding="utf-8") as f:
        lines = f.readlines()

    for line in tqdm(lines, desc=f"🖼️ {chart_type}"):
        if random.random() > 0.5:
            continue
        entry = json.loads(line)
        print(f"🔍 Processing image {entry['id']}...")
        image_id = entry["id"]
        image_path = os.path.join(cfg["img_dir"], os.path.basename(entry["image"]).replace("\\", "/"))
        points = random.sample(entry["points"], min(1, len(entry["points"])))

        try:
            image = Image.open(image_path).convert("RGB")
        except Exception as e:
            print(f"❌ Cannot open image {image_id}: {e}")
            continue

        for point in points:
            x_val = point["x"]
            y_true = point["y"]
            start_time = time.perf_counter()
            y_pred_raw = query_model(image, x_val)
            elapsed_time = time.perf_counter() - start_time

            # Try to parse float
            try:
                y_pred = float(y_pred_raw.split()[0].replace(",", "."))
            except Exception:
                y_pred = None

            abs_error = abs(y_pred - y_true) if y_pred is not None else None
            results.append({
                "chart_type": chart_type,
                "image_id": image_id,
                "x": x_val,
                "y_true": y_true,
                "y_pred": y_pred,
                "abs_error": abs_error,
                "raw_output": y_pred_raw,
                "inference_time": elapsed_time
            })

    df = pd.DataFrame(results)
    df.to_csv(cfg["output_csv"], index=False)
    print(f"✅ Saved {chart_type} results to {cfg['output_csv']}")

📁 Processing bar charts...


🖼️ bar:   0%|          | 0/20 [00:00<?, ?it/s]

🔍 Processing image 48e6a097-e192-4e83-9d09-cc4f23efec9b...


The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
🖼️ bar:  10%|█         | 2/20 [02:55<26:23, 87.97s/it]

🔍 Processing image ba23b1c0-4e6c-4a67-8d44-353bcf4c34a5...


...

🔍 Processing image 5dd3bf98-64c0-4209-894a-ad5dfa9b3674...


🖼️ scatter:  88%|████████▊ | 44/50 [19:31<02:21, 23.55s/it]The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


🔍 Processing image ec73f558-ece3-41d6-b711-25f873a94743...


🖼️ scatter: 100%|██████████| 50/50 [20:18<00:00, 24.37s/it]

✅ Saved scatter results to gemma3_4b_results_scatter.csv
